In [1]:
import sqlite3
from datetime import datetime
from itertools import groupby

In [117]:
# Подключение к базе данных (файл будет создан, если не существует)
conn = sqlite3.connect('db/trading_bot.db')

# Создание курсора
cursor = conn.cursor()

# Создание таблицы сделок
cursor.execute('''
CREATE TABLE IF NOT EXISTS deals (
    id INTEGER PRIMARY KEY,
    algorithm_name TEXT NOT NULL,
    type INTEGER NOT NULL,
    instrument TEXT NOT NULL,
    datetime DATETIME DEFAULT CURRENT_TIMESTAMP,
    price REAL NOT NULL,
    commission REAL NOT NULL,
    total REAL NOT NULL
)
''')

# Создание индексов
cursor.execute('CREATE INDEX IF NOT EXISTS idx_instrument_datetime ON deals (algorithm_name, datetime)')
cursor.execute('CREATE INDEX IF NOT EXISTS idx_datetime ON deals (datetime)')

# Закрытие соединения
conn.close()

In [232]:
# Очистка базы. Нужно раскомментировать код
conn = sqlite3.connect('db/trading_bot.db')
cursor = conn.cursor()

# Создание таблицы сделок
# cursor.execute('''
# DELETE FROM deals
# ''')

conn.close()

In [189]:
def get_daily_totals(date=None):
    if date is None:
        date = datetime.now().strftime('%Y-%m-%d')
        
    conn = sqlite3.connect('db/trading_bot.db')
    cursor = conn.cursor()
    
    # Получаем аггрегированные данные по дням и algorithm_name
    cursor.execute('''
    SELECT 
        algorithm_name, 
        strftime('%Y-%m-%d', SUBSTR(datetime, 1, 19)) AS day,
        SUM(total) AS daily_total,
        count(*) AS cnt
    FROM deals
    WHERE date(SUBSTR(datetime, 1, 19)) = ?
    GROUP BY algorithm_name, day
    ORDER BY algorithm_name, day
    ''', (date,))
    
    aggregated_results = cursor.fetchall()
    
    # Итерация по аггрегированным результатам и выполнение дополнительного запроса для каждой пары algorithm_name, day
    for algorithm_name, day, daily_total, cnt in aggregated_results:
        cursor.execute('''
        SELECT type, total FROM deals
        WHERE algorithm_name = ? AND date(SUBSTR(datetime, 1, 19)) = date(?)
        ORDER BY datetime DESC
        LIMIT 1
        ''', (algorithm_name, day))
        
        last_operation_type, last_total = cursor.fetchone()
        
        # Если тип последней операции равен 1, вычитаем её total из итогового результата
        if last_operation_type == 1:           
            adjusted_total = daily_total - last_total
            print(f"День: {day}, {algorithm_name}, "
                  f"Подытог: {round(adjusted_total, 2)}, "
                  f"Записей: {cnt}")
        else:
            print(f"День: {day}, {algorithm_name}, "
                  f"Итог: {round(daily_total, 2)}, "
                  f"Записей: {cnt}")
    
    conn.close()

In [54]:
def add_deal(algorithm_name, type, instrument, price, commission, total):
    conn = sqlite3.connect('db/trading_bot.db')
    cursor = conn.cursor()
    cursor.execute('''
    INSERT INTO deals (algorithm_name, type, instrument, price, commission, total)
    VALUES (?, ?, ?, ?, ?, ?)
    ''', (algorithm_name, type, instrument, price, commission, total))
    conn.commit()
    conn.close()

In [3]:
# добавление записи
# add_deal(f"RNFT_v2_c4_s2.py", 2, 'RNFT', 162.6, 0.9, 162.6-0.9)

In [147]:
def get_hourly_totals(date=None):
    if date is None:
        date = datetime.now().strftime('%Y-%m-%d')  # Текущая дата в формате строки

    conn = sqlite3.connect('db/trading_bot.db')
    cursor = conn.cursor()

    # Получаем данные за указанный день
    cursor.execute('''
    SELECT 
        algorithm_name,
        strftime('%H', SUBSTR(datetime, 1, 19)) AS hour,
        total,
        type
    FROM deals
    WHERE date(SUBSTR(datetime, 1, 19)) = ?
    ORDER BY algorithm_name, hour, datetime, id
    ''', (date,))

    results = cursor.fetchall()

    # группировка по имени алгоритма
    for algorithm_name, algorithm_group in groupby(results, key=lambda x: x[0]):
        algorithm_list = list(algorithm_group)
        
        print(f"Algorithm: {algorithm_name}")
        
        sum_price = 0
        prev_show = None
        # группировка часам
        for hour, hour_group in groupby(algorithm_list, key=lambda x: x[1]):
            hour_list = list(hour_group)
            
            hour_sum_price = 0
            operation_type = 0
            operation_total = 0 
            for _, _, operation_total, operation_type in hour_list:
                hour_sum_price += operation_total
            
            diff = hour_sum_price
            sum_price += hour_sum_price
            
            show_sum = sum_price
            # откидываем последнюю операцию, но только для локального вывода
            if operation_type == 1:
                show_sum -= operation_total
                # diff -= operation_total
                
            print(f"  {hour}: {round(show_sum, 2)} "
                  f"({round(show_sum - prev_show, 2) if prev_show is not None else '0'})")

            prev_show = show_sum                
            
    conn.close()

In [231]:
get_daily_totals()

День: 2024-03-15, v2_c3_s5, Итог: -0.38, Записей: 2
День: 2024-03-15, v2_c3_s5_time_drop, Подытог: 0.12, Записей: 3
День: 2024-03-15, v2_c4_s4_no_2steps, Итог: -0.86, Записей: 4
День: 2024-03-15, v2_c4_s5, Подытог: 1.84, Записей: 5
День: 2024-03-15, v2_c4_s5_no_2steps, Подытог: 1.32, Записей: 3
День: 2024-03-15, v2_c4_s5_time_drop, Подытог: 1.84, Записей: 5
День: 2024-03-15, v2_c4_s6, Подытог: 0.62, Записей: 3


In [222]:
get_hourly_totals()  # Для текущего дня
# get_hourly_totals('2023-01-01')  # Для конкретной даты

Algorithm: v2_c3_s5
  12: 0.0 (0)
Algorithm: v2_c3_s5_time_drop
  11: 0.12 (0)
  12: 0.12 (0.0)
Algorithm: v2_c4_s4_no_2steps
  10: 0.0 (0)
  11: -0.48 (-0.48)
  12: -0.86 (-0.38)
Algorithm: v2_c4_s5
  10: 0.62 (0)
  11: 0.62 (0.0)
  12: 1.84 (1.22)
Algorithm: v2_c4_s5_no_2steps
  11: 0.0 (0)
  12: 1.32 (1.32)
Algorithm: v2_c4_s5_time_drop
  10: 0.62 (0)
  11: 0.62 (0.0)
  12: 1.84 (1.22)
Algorithm: v2_c4_s6
  10: 0.62 (0)
  12: 0.62 (0.0)
